# Выбор семейства распределений в наивном байесе

В этом задании вы сможете немного расслабиться после реализации случайного леса и градиентного бустинга по мотивам материалов прошлой недели. Все, что нужно будет делать — запускать методы из sklearn. Вам предлагается выяснить, какое распределение лучше использовать в наивном байесовском классификаторе в зависимости от вида признаков.

Загрузите датасеты digits и breast_cancer из sklearn.datasets. Выведите несколько строчек из обучающих выборок и посмотрите на признаки. С помощью sklearn.cross_validation.cross_val_score c настройками по умолчанию и вызова метода mean() у возвращаемого этой функцией numpy.ndarray, сравните качество работы наивных байесовских классификаторов на этих двух датасетах. Для сравнения предлагается использовать BernoulliNB, MultinomialNB и GaussianNB. Насколько полученные результаты согласуются с рекомендациями из лекций?

Два датасета, конечно, еще не повод делать далеко идущие выводы, но при желании вы можете продолжить исследование на других выборках (например, из UCI репозитория).

Для сдачи задания, ответьте на приведенные ниже вопросы.

In [28]:
from sklearn import datasets
import numpy as np
import pandas as pd
import math
from sklearn import model_selection, metrics
from sklearn.model_selection import cross_val_score as cv
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
%pylab inline

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [24]:
x_d = pd.DataFrame(data = datasets.load_digits().data,
                   columns=list(datasets.load_digits().feature_names))
y_d = pd.DataFrame(datasets.load_digits().target)


x_bc = pd.DataFrame(data = datasets.load_breast_cancer().data,
                   columns = list(datasets.load_breast_cancer().feature_names))
y_bc = pd.DataFrame(datasets.load_breast_cancer().target)

In [25]:
x_d.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [26]:
x_bc.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [32]:
clf_B = BernoulliNB()
clf_M = MultinomialNB()
clf_G = GaussianNB()
cross_val_B_d = cv(clf_B, x_d, y_d,  cv = 10).mean()
cross_val_B_bc = cv(clf_B, x_bc, y_bc,  cv = 10).mean()

cross_val_M_d = cv(clf_M, x_d, y_d,  cv = 10).mean()
cross_val_M_bc = cv(clf_M, x_bc, y_bc,  cv = 10).mean()

cross_val_G_d = cv(clf_G, x_d, y_d,  cv = 10).mean()
cross_val_G_bc = cv(clf_G, x_bc, y_bc,  cv = 10).mean()

In [35]:
print(f'Cross val score for digits:\n\
Bernoulli: {cross_val_B_d}; \n\
Multinomial: {cross_val_M_d}; \n\
Gaussian: {cross_val_G_d}')

print(f'\nCross val score for breast_cancer:\n\
Bernoulli: {cross_val_B_bc}; \n\
Multinomial: {cross_val_M_bc}; \n\
Gaussian: {cross_val_G_bc}')

Cross val score for digits:
Bernoulli: 0.8325170701427685; 
Multinomial: 0.8797858472998138; 
Gaussian: 0.8113904407200497

Cross val score for breast_cancer:
Bernoulli: 0.6274122807017544; 
Multinomial: 0.8927944862155389; 
Gaussian: 0.9367794486215537


## Вопрос 1

Каким получилось максимальное качество классификации на датасете breast_cancer?

In [44]:
def write_answer(ans, text):
    with open(text, "w") as fout:
        fout.write(str(ans))
        
write_answer(cross_val_G_bc, 'nb_ans1.txt')

## Вопрос 2

Каким получилось максимальное качество классификации на датасете digits

In [43]:
write_answer(cross_val_M_d, 'nb_ans2.txt')

## Вопрос 3

Выберите верные утверждения и запишите их номера через пробел (в порядке возрастания номера):

1) На вещественных признаках лучше всего сработал наивный байесовский классификатор с распределением Бернулли

2) На вещественных признаках лучше всего сработал наивный байесовский классификатор с мультиномиальным распределением

3) Мультиномиальное распределение лучше показало себя на выборке с целыми неотрицательными значениями признаков

4) На вещественных признаках лучше всего сработало нормальное распределение

In [45]:
ans = '3 4'
write_answer(ans, 'nb_ans3.txt')